In [23]:
import numpy as np
import random
from fractions import Fraction
from Commands import dec2int, bernoulli, control, order_parameter
from MonteCarloCommands import energy, boltzmann_probability
import matplotlib.pyplot as plt




In [24]:

length = 100
random.seed(10)
times = 5
low_prob = 00
high_prob = 100 + 1
Beta =.1
K = 1 
num_chains = 2
spacing = 10
num_points = int(np.ceil((high_prob-low_prob)/spacing))




In [26]:
rep1 = random.getrandbits(length)
rep2 = random.getrandbits(length)
print(bin(rep1),bin(rep2))
lattice = [format(rep1, '0'+str(length)+'b'), format(rep2, '0'+str(length)+'b')]
                       #format(rep3, '0'+str(length)+'b')]
for repeat in range((length**2)):
    for y_pos in range(0, length): # this section takes about 1/100 s at length = 100, accounts for most of the run time
        for x_pos in range(0, num_chains):
            E_i = energy(x_pos, y_pos, lattice, num_chains, length, K, True)
            old = lattice[x_pos]
            # noinspection PyTypeChecker
            lattice[x_pos] = format(int(lattice[x_pos], base=2) ^ (1 << (length - y_pos - 1)), '0' + str(length) + 'b')
            E_f = energy(x_pos, y_pos, lattice, num_chains, length, K, True)
            if random.random() <= boltzmann_probability(E_i, E_f, Beta):
                pass
            else:
                lattice[x_pos] = old
print(lattice[0],lattice[1])

0b1010011011101011001110110111101001011100001011001001010000101101101111001011010111101001001111010100 0b10101110111101110000000111111111011001111010001011011011100001001100111010110111110100101011110100
1111011101110011101100001010010010011011101110101001010101011110111101110101100000111001110100111110 1011010110100111000100101001001011100001001001010100011100101110100010010001101010000100101011111100
